# Moderated Chat application with `AmazonComprehendModerationChain`
---

Install the necessary libraries. We use [Gradio](https://www.gradio.app/) for the Chat UI interface.

<div class="alert alert-block alert-info"> <b>NOTE:</b> You must use a Python3 kernel to run this notebook.</div>

In [ ]:
%pip install -U gradio boto3 nltk

In [ ]:
%pip install -U langchain langchain_experimental huggingface_hub

## Sample Chat application with HuggingFace Hub 
---

In [ ]:
import boto3
from getpass import getpass
import os

comprehend_client = boto3.client('comprehend', region_name='us-east-2')

Get your API Key from Huggingface hub - https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

<div class="alert alert-block alert-info"> <b>NOTE:</b> You must have access to Amazon Comprehend Toxicity and Intent APIs to be able to use the code below as is. Please see the blog for more details on how to get early access to these APIs. If you do not have access to these APIs, you can still use this chat demo with PII check only by using a configuration with PII filter only. Refer to the "<b>Using moderation_config to customize your moderation</b>" section in the previous notebook for more details.</div>

We build a Gradio chat application demo using HuggingFace Hub model. We use `AmazonComprehendModerationChain` with default configuration.

In [ ]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain_experimental.comprehend_moderation import AmazonComprehendModerationChain
import gradio as gr
import os

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], label="GenAI Chat", height=400)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):    
        repo_id = "google/flan-t5-xxl" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
        
        template = """Question: {question}

        Answer:"""

        prompt = PromptTemplate(template=template, input_variables=["question"])
        llm = HuggingFaceHub(
            repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 256}
        )
        llm_chain = LLMChain(prompt=prompt, llm=llm)
        

        comprehend_moderation = AmazonComprehendModerationChain(client=comprehend_client, 
                                                                verbose=False)
        
        try:
            chain = (prompt 
                     | comprehend_moderation 
                     | {llm_chain.input_keys[0]: lambda x: x['output'] }  
                     | llm_chain 
                     | { "input": lambda x: x['text'] } 
                     | comprehend_moderation 
                    )
            response = chain.invoke({"question": message})
            chat_history.append((message, response['output']))
        except Exception as e:
            error_msg = f"<b style='color:red;'>Error: {str(e)}</b>"
            chat_history.append((message, error_msg))

        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

## Chatbot with `AmazonComprehendModerationChain` with configurations
---

### Chat with ALLOW action

With PII moderation only to allow text but mask any SSN.

Sample prompt:

```
What is John Doe's address, phone number and SSN from the following text?

John Doe, a resident of 1234 Elm Street in Springfield, recently celebrated his birthday on January 1st. Turning 43 this year, John reflected on the years gone by. He often shares memories of his younger days with his close friends through calls on his phone, (555) 123-4567. Meanwhile, during a casual evening, he received an email at johndoe@example.com reminding him of an old acquaintance's reunion. As he navigated through some old documents, he stumbled upon a paper that listed his SSN as 123-45-6789, reminding him to store it in a safer place.
```

In [ ]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain_experimental.comprehend_moderation import (AmazonComprehendModerationChain,
                                                          BaseModerationActions, 
                                                          BaseModerationFilters)
import gradio as gr
import os

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], label="GenAI Chat", height=400)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):    
        repo_id = "google/flan-t5-xxl" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
        
        template = "Please answer to the following question: {question}"

        prompt = PromptTemplate(template=template, input_variables=["question"])
        llm = HuggingFaceHub(
            repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 256}
        )
        llm_chain = LLMChain(prompt=prompt, llm=llm)
        
        moderation_config = { 
                "filters":[ 
                        BaseModerationFilters.PII
                ],
                "pii":{ 
                        "action": BaseModerationActions.ALLOW, 
                        "threshold":0.5, 
                        "labels":["SSN"],
                        "mask_character": "X"
                }
        }

        comprehend_moderation = AmazonComprehendModerationChain(client=comprehend_client, #optional
                                                                moderation_config=moderation_config, 
                                                                verbose=False)
        
        try:
            chain = (prompt 
                     | comprehend_moderation 
                     | {llm_chain.input_keys[0]: lambda x: x['output'] }  
                     | llm_chain 
                     | { "input": lambda x: x['text'] } 
                     | comprehend_moderation 
                    )
            response = chain.invoke({"question": message})
            chat_history.append((message, response['output']))
        except Exception as e:
            error_msg = f"<b style='color:red;'>Error: {str(e)}</b>"
            chat_history.append((message, error_msg))

        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

---
### Chat with STOP action

With PII moderation only to block text with any `SSN` or `PHONE_NUMBER`.

Sample prompt:

```
What is John Doe's address, phone number and SSN from the following text?

John Doe, a resident of 1234 Elm Street in Springfield, recently celebrated his birthday on January 1st. Turning 43 this year, John reflected on the years gone by. He often shares memories of his younger days with his close friends through calls on his phone, (555) 123-4567. Meanwhile, during a casual evening, he received an email at johndoe@example.com reminding him of an old acquaintance's reunion. As he navigated through some old documents, he stumbled upon a paper that listed his SSN as 123-45-6789, reminding him to store it in a safer place.
```

In [ ]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain_experimental.comprehend_moderation import (AmazonComprehendModerationChain,
                                                          BaseModerationActions, 
                                                          BaseModerationFilters)
import gradio as gr
import os

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], label="GenAI Chat", height=400)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):    
        repo_id = "google/flan-t5-xxl" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
        
        template = "Please answer to the following question: {question}"

        prompt = PromptTemplate(template=template, input_variables=["question"])
        llm = HuggingFaceHub(
            repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 256}
        )
        llm_chain = LLMChain(prompt=prompt, llm=llm)
        
        moderation_config = { 
                "filters":[ 
                        BaseModerationFilters.PII
                ],
                "pii":{ 
                        "action": BaseModerationActions.STOP, 
                        "threshold":0.5, 
                        "labels":["SSN", "PHONE_NUMBER"],
                        "mask_character": "X"
                }
        }

        comprehend_moderation = AmazonComprehendModerationChain(client=comprehend_client, #optional
                                                                moderation_config=moderation_config, 
                                                                verbose=False)
        
        try:
            chain = (prompt 
                     | comprehend_moderation 
                     | {llm_chain.input_keys[0]: lambda x: x['output'] }  
                     | llm_chain 
                     | { "input": lambda x: x['text'] } 
                     | comprehend_moderation 
                    )
            response = chain.invoke({"question": message})
            chat_history.append((message, response['output']))
        except Exception as e:
            error_msg = f"<b style='color:red;'>Error: {str(e)}</b>"
            chat_history.append((message, error_msg))

        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()